<a href="https://colab.research.google.com/github/csabiu/astronomy/blob/main/Astro_distances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install some packages and import them

In [ ]:
pip install astroquery

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import (SkyCoord, Distance, Galactic, 
                                 EarthLocation, AltAz)
import astropy.coordinates as coord
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import AltAz
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file
from astroquery.gaia import Gaia

Define a target object NGC188 - an open star cluster 

different ways of defining in astropy

In [ ]:
ngc188_center = SkyCoord.from_name('NGC 188')
ngc188_center

Lets search the GAIA database for stars with 0.5 degrees of NGC188 center

In [ ]:
Gaia.ROW_LIMIT = 10000  # Set the row limit for returned data

job = Gaia.cone_search_async(ngc188_center, radius=0.5*u.deg)
ngc188_table = job.get_results()

# only keep stars brighter than G=19 magnitude
ngc188_table = ngc188_table[ngc188_table['phot_g_mean_mag'] < 19*u.mag]

In [ ]:
ngc188_table.colnames

In [ ]:
cols = [
    'source_id',
    'ra',
    'dec',
    'parallax',
    'parallax_error',
    'pmra',
    'pmdec',
    'radial_velocity',
    'phot_g_mean_mag',
    'phot_bp_mean_mag',
    'phot_rp_mean_mag'
]
ngc188_table[cols].write('gaia_results.fits', overwrite=True)



In [ ]:
ngc188_table = QTable.read('gaia_results.fits')
len(ngc188_table)

With the table of Gaia data we retrieved above for stars around NGC 188, ngc188_table, we also have parallax measurements for each star. For a precisely-measured parallax ϖ
, the distance d to a star can be obtained approximately as d≈1/ϖ. This only really works if the parallax error is small relative to the parallax, so if we want to use these parallaxes to get distances we first have to filter out stars that have low signal-to-noise parallaxes:

In [ ]:
parallax_snr = ngc188_table['parallax'] / ngc188_table['parallax_error']
ngc188_table_3d = ngc188_table[parallax_snr > 10] 
len(ngc188_table_3d)

In [ ]:
parallax=ngc188_table_3d['parallax'].filled(np.nan)
parallax

In [ ]:
Distance(parallax=1*u.mas)

In [ ]:
gaia_dist=Distance(parallax=parallax)
gaia_dist

In [ ]:
ra=ngc188_table_3d['ra']
dec=ngc188_table_3d['dec']

ngc188_coords_3d = SkyCoord(ra=ngc188_table_3d['ra'], 
                            dec=ngc188_table_3d['dec'],
                            distance=gaia_dist)
ngc188_coords_3d



converting parallax angle to distance

In [ ]:


fig, ax = plt.subplots(figsize=(6.5, 5.2),
                       constrained_layout=True)
cs = ax.scatter(ngc188_coords_3d.ra.degree, 
                ngc188_coords_3d.dec.degree, 
                c=ngc188_coords_3d.distance.kpc, 
                s=5, vmin=1.5, vmax=2.5, cmap='twilight')
cb = fig.colorbar(cs)
cb.set_label(f'distance [{u.kpc:latex_inline}]')

ax.set_xlabel('RA [deg]')
ax.set_ylabel('Dec [deg]')

ax.set_title('Gaia DR2 sources near NGC 188', fontsize=18)



Lets define the 3D coordinate to the cluster centre and then find the distance of each Gaia star to the NGC188 centre, then replot the above with a shifted distance (colorbar)

In [ ]:
ngc188_center_3d = SkyCoord(12.11*u.deg, 85.26*u.deg,
                            distance=1.96*u.kpc)

sep3d = ngc188_coords_3d.separation_3d(ngc188_center_3d)
sep3d

In [ ]:

fig, ax = plt.subplots(figsize=(6.5, 5.2),
                       constrained_layout=True)
cs = ax.scatter(ngc188_coords_3d.ra.degree, 
                ngc188_coords_3d.dec.degree, 
                c=sep3d.kpc, 
                s=5, vmin=0.0, vmax=2.0, cmap='hot')
cb = fig.colorbar(cs)
cb.set_label(f'distance [{u.kpc:latex_inline}]')

ax.set_xlabel('RA [deg]')
ax.set_ylabel('Dec [deg]')

ax.set_title('Gaia DR2 sources near NGC 188', fontsize=18)
